# Prediction of forest fire in Portugal

Through a regression model, we will predict the damaged areas by forest fire on a  region of Portugal. The prediction consist on whether a forest fire can be affected by a fire, and if it is, how many hectareas can be the fire affect.


## 1. Data

The data is downloaded from UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Forest+Fires
Data Set Information:

#### Prior results
In [Cortez and Morais, 2007], the output 'area' was first transformed with a ln(x+1) function. Then, several Data Mining methods were applied. After fitting the models, the outputs were post-processed with the inverse of the ln(x+1) transform. Four different input setups were used. The experiments were conducted using a 10-fold (cross-validation) x 30 runs. Two regression metrics were measured: MAD and RMSE. A Gaussian support vector machine (SVM) fed with only 4 direct weather conditions (temp, RH, wind and rain) obtained the best MAD value: 12.71 +- 0.01 (mean and confidence interval within 95% using a t-student distribution). The best RMSE was attained by the naive mean predictor. An analysis to the regression error curve (REC) shows that the SVM model predicts more examples within a lower admitted error. In effect, the SVM model predicts better small fires, which are the majority. 

### Features

1. `X `- x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. `Y `- y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. `month` - month of the year: 'jan' to 'dec'
4. `day` - day of the week: 'mon' to 'sun'
5. `FFMC` - FFMC (Fine Fuel Moisture Code) index from the FWI system: 18.7 to 96.20
6. `DMC` - DMC (Dufe moisture Code) index from the FWI system: 1.1 to 291.3
7. `DC` - DC (Drought Code) index from the FWI (Fire Weather Index) system: 7.9 to 860.6
8. `ISI` - ISI (Initial Spread Index) index from the FWI system: 0.0 to 56.10
9. `temp` - temperature in Celsius degrees: 2.2 to 33.30
10. `RH` - relative humidity in %: 15.0 to 100
11. `wind` - wind speed in km/h: 0.40 to 9.40
12. `rain` - outside rain in mm/m2 : 0.0 to 6.4
13. `area` - the burned area of the forest (in ha): 0.00 to 1090.84 

`FFMC`: Denotes the moisture content surface litter and influences ignition and fire spread. 

`DMC and DC`: Represent the moisture content of shallow and deep organic layers, which affect fire intensity.

`ISI`: Score that correlates with fire velocity spread.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# Transforming categorical data month into numbers
def cat_trans(X):
    X['month'] = X['month'].replace('jan', 1)
    X['month'] = X['month'].replace('feb', 2)
    X['month'] = X['month'].replace('mar', 3)
    X['month'] = X['month'].replace('apr', 4)
    X['month'] = X['month'].replace('may', 5)
    X['month'] = X['month'].replace('jun', 6)
    X['month'] = X['month'].replace('jul', 7)
    X['month'] = X['month'].replace('aug', 8)
    X['month'] = X['month'].replace('sep', 9)
    X['month'] = X['month'].replace('oct', 10)
    X['month'] = X['month'].replace('nov', 11)
    X['month'] = X['month'].replace('dec', 12)
#     X['day'] = X['day'].replace('sun', 1)
#     X['day'] = X['day'].replace('mon', 2)
#     X['day'] = X['day'].replace('tue', 3)
#     X['day'] = X['day'].replace('wed', 4)
#     X['day'] = X['day'].replace('thu', 5)
#     X['day'] = X['day'].replace('fri', 6)
#     X['day'] = X['day'].replace('sat', 7)
    
    return X

In [3]:
def model_test(model, data):
    np.random.seed(42)
                  
    data.drop(['ISI', 'wind', 'rain', 'day', 'X', 'Y'], axis=1, inplace=True)
    
    
    X = data.drop('area', axis=1)
    y = data['area']
    
    X = cat_trans(X)
    
    #Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Scaling the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train = np.array(y_train).reshape(-1,1)
    y_test = np.array(y_test).reshape(-1,1)
    y_train_scaled = scaler.fit_transform(y_train)
    y_test_scaled = scaler.fit_transform(y_test)
    
    # Fitting the model
    model.fit(X_train_scaled, y_train_scaled)
    
    # Making predictions
    pred = model.predict(X_test_scaled)
    
    pred_u = scaler.inverse_transform(np.array(pred).reshape(-1, 1))
    y_test_u = scaler.inverse_transform(np.array(y_test_scaled).reshape(-1, 1))
    diff = np.array(pred_u) - np.array(y_test_u)
    
    summary_diff = pd.DataFrame(np.concatenate((pred_u, y_test_u, diff), axis=1), columns=["Predictions", "Real", "Difference"])
                                       
    return print(
    "R2 square: ", r2_score(y_test_scaled, pred), " Mean Absolute Error: ", mean_absolute_error(y_test_scaled, pred),
        " Mean Squared Error: ", mean_squared_error(y_test_scaled, pred), "\n", 
     summary_diff
        
    )

### Using the original dataset

In [4]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
data = pd.read_csv('forestfires.csv')
model_test(model, data)

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 square:  -0.873895188572108  Mean Absolute Error:  0.03537509458378968  Mean Squared Error:  0.014919877977318816 
      Predictions   Real  Difference
0       7.218438   0.00    7.218438
1       0.419658   0.00    0.419658
2     552.401755   1.23  551.171755
3      18.339457   1.46   16.879457
4       0.989436   1.95   -0.960564
..           ...    ...         ...
125     8.945747   0.00    8.945747
126    15.031033   5.80    9.231033
127    14.847029   3.93   10.917029
128    12.817013   0.00   12.817013
129    16.209904  22.03   -5.820096

[130 rows x 3 columns]


### Tuning the model by hyperparameter

In [7]:
from sklearn.model_selection import RandomizedSearchCV
np.random.seed(42)
data = pd.read_csv('forestfires.csv')
#data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop(['ISI', 'wind', 'rain', 'day', 'X', 'Y'], axis=1, inplace=True)

X = data.drop('area', axis=1)
y = data['area']

X = cat_trans(X)

#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scaling the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.fit_transform(y_test)


# Different Support Vector Regressor hyperparameter
rf_grid = {"loss":['ls', 'lad', 'huber', 'quantile'],
           "learning_rate": [0.1, 0.2, 0.3],
           "n_estimators": np.arange(0,100,10),
           "criterion":['friedman_mse', 'mse', 'mae'],
           "min_samples_split":[2,3,4,5,6,7,8],
           "min_samples_leaf":[1,2,3,4,5,6,7,8],
           "max_features":['auto', 'sqrt', 'log2']
           
    

          }  # limiting the maximum to save time

# Instantiate RandomizedSearchCV model
gd_model = RandomizedSearchCV(GradientBoostingRegressor(),
                              param_distributions=rf_grid,
                              n_iter = 100, 
                              cv=20,
                              verbose=True
                             )
# Fit the RandomizedSearchCV model
gd_model.fit(X_train_scaled, y_train_scaled)

#print(tree_model.best_params_)
gd.best_params_

Fitting 20 folds for each of 100 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\model_selection\_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: n_estimators must be greater than 0 but was 0

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\model_selection\_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: n_estimators must be greater tha

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

...

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-pack

NameError: name 'gd' is not defined

In [9]:
gd_model.best_params_

{'n_estimators': 10,
 'min_samples_split': 5,
 'min_samples_leaf': 7,
 'max_features': 'auto',
 'loss': 'huber',
 'learning_rate': 0.1,
 'criterion': 'mae'}

In [25]:
model = GradientBoostingRegressor(n_estimators=50, min_samples_split=3, min_samples_leaf=7,
                                 max_features='auto', loss='huber', learning_rate=0.1, criterion='friedman_mse')
data = pd.read_csv('forestfires.csv')
model_test(model, data)

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 square:  0.002497196063059759  Mean Absolute Error:  0.01757996856563507  Mean Squared Error:  0.007942077127650321 
      Predictions   Real  Difference
0       4.240471   0.00    4.240471
1       8.595132   0.00    8.595132
2      16.434740   1.23   15.204740
3       7.718156   1.46    6.258156
4       1.330689   1.95   -0.619311
..           ...    ...         ...
125     6.030412   0.00    6.030412
126     9.026371   5.80    3.226371
127     7.918228   3.93    3.988228
128     7.937644   0.00    7.937644
129    14.877851  22.03   -7.152149

[130 rows x 3 columns]


## Dataset hazard with outlier


In [42]:
model = GradientBoostingRegressor(n_estimators=50, min_samples_split=3, min_samples_leaf=2,
                                 max_features='auto', loss='huber', learning_rate=0.1, criterion='mse')
data = pd.read_csv('forest_fires_affected_areas.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
model_test(model, data)

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 square:  0.2271201006190654  Mean Absolute Error:  0.03105013043548158  Mean Squared Error:  0.011401515208529485 
     Predictions   Real  Difference
0     16.448550   3.35   13.098550
1     13.353824   6.43    6.923824
2     32.576746  11.16   21.416746
3     11.854865  42.87  -31.015135
4     11.545551   2.93    8.615551
..          ...    ...         ...
63     9.552803   7.19    2.362803
64    17.750598   8.12    9.630598
65    20.544560   2.87   17.674560
66    18.207287  39.35  -21.142713
67    15.140803   1.64   13.500803

[68 rows x 3 columns]


### Result
The best result we have obtained so far

## Dataset hazard without outlier

In [60]:
model = GradientBoostingRegressor(n_estimators=100, min_samples_split=3, min_samples_leaf=8,
                                 max_features='auto', loss='huber', learning_rate=0.1, criterion='friedman_mse')
data = pd.read_csv('forestfires_affected_areas_no_outliers.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
model_test(model, data)

C:\Users\sandra\Desktop\Miniconda_projects\projects\forest_fires\env\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 square:  -0.05951930243379144  Mean Absolute Error:  0.08330703910165815  Mean Squared Error:  0.041002290542827405 
     Predictions    Real  Difference
0      3.414530  185.76 -182.345470
1      4.929781   34.36  -29.430219
2      5.763240    2.53    3.233240
3      9.818157    3.30    6.518157
4      6.428083   82.75  -76.321917
..          ...     ...         ...
62     6.510590    2.03    4.480590
63     4.235246    1.64    2.595246
64    11.831631   49.59  -37.758369
65    11.268223    2.07    9.198223
66     5.975200    1.63    4.345200

[67 rows x 3 columns]


### Results
Without outliers, the model does not do any better.